## 1. Inicializações para o Laboratório

Instalação de todas as dependências necessárias para o pipeline completo.

In [ ]:
# Detectar ambiente (Colab ou Local)
import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔧 Ambiente: Google Colab")
else:
    print("🔧 Ambiente: Jupyter Local")

# Variáveis globais
SPARK_VERSION = "3.5.0"
HADOOP_VERSION = "3"
KAFKA_VERSION = "3.6.0"
SCALA_VERSION = "2.12"

print(f"📦 Versões: Spark {SPARK_VERSION}, Kafka {KAFKA_VERSION}, Scala {SCALA_VERSION}")

In [ ]:
# Instalação de dependências Python
!pip install -q pyspark=={SPARK_VERSION}
!pip install -q kafka-python
!pip install -q wordcloud matplotlib pandas numpy

print("✅ Dependências Python instaladas")

In [ ]:
# Download do Kafka (necessário para broker local)
import subprocess
import tarfile
import urllib.request

KAFKA_DIR = f"/tmp/kafka_{SCALA_VERSION}-{KAFKA_VERSION}"

if not os.path.exists(KAFKA_DIR):
    print("📥 Baixando Apache Kafka...")
    kafka_url = f"https://archive.apache.org/dist/kafka/{KAFKA_VERSION}/kafka_{SCALA_VERSION}-{KAFKA_VERSION}.tgz"
    kafka_tgz = f"/tmp/kafka.tgz"
    
    urllib.request.urlretrieve(kafka_url, kafka_tgz)
    
    print("📦 Extraindo Kafka...")
    with tarfile.open(kafka_tgz, "r:gz") as tar:
        tar.extractall("/tmp/")
    
    os.remove(kafka_tgz)
    print(f"✅ Kafka instalado em {KAFKA_DIR}")
else:
    print(f"✅ Kafka já existe em {KAFKA_DIR}")

## 2. Configuração de Mecanismos para Visualização de Resultados

Preparação do ambiente para coleta e exibição dos resultados em tempo real.

In [ ]:
# Configuração de diretórios
WORK_DIR = "/tmp/spark_lab"
CHECKPOINT_DIR = f"{WORK_DIR}/checkpoints"
RESULTS_DIR = f"{WORK_DIR}/results"

os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"📁 Diretório de trabalho: {WORK_DIR}")
print(f"📁 Checkpoints: {CHECKPOINT_DIR}")
print(f"📁 Resultados: {RESULTS_DIR}")

In [ ]:
# Imports para visualização
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
import time
from collections import Counter

# Configuração de plots
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Bibliotecas de visualização configuradas")

## 3. Instalação e Configuração do Apache Spark

Setup do Spark com suporte a Kafka Structured Streaming.

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

# Configuração do Spark com pacotes Kafka
conf = SparkConf()
conf.set("spark.jars.packages", 
         f"org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION}")
conf.set("spark.sql.streaming.checkpointLocation", CHECKPOINT_DIR)
conf.set("spark.driver.memory", "2g")
conf.set("spark.executor.memory", "2g")

# Criar sessão Spark
spark = SparkSession.builder \
    .appName("B2_WordCount_Streaming") \
    .config(conf=conf) \
    .master("local[*]") \
    .getOrCreate()

# Configurar nível de log
spark.sparkContext.setLogLevel("WARN")

print(f"✅ Spark Session criada: {spark.version}")
print(f"📊 Spark UI: http://localhost:4040")

## 4. Instalação e Configuração do Kafka

Inicialização do Kafka broker e criação dos tópicos de entrada e saída.

In [ ]:
import subprocess
import signal
import atexit

# Processos do Kafka
zookeeper_process = None
kafka_process = None

def cleanup_kafka():
    """Finaliza processos Kafka ao encerrar notebook"""
    global zookeeper_process, kafka_process
    
    if kafka_process:
        kafka_process.terminate()
        kafka_process.wait()
    
    if zookeeper_process:
        zookeeper_process.terminate()
        zookeeper_process.wait()
    
    print("🛑 Kafka e Zookeeper finalizados")

# Registrar cleanup
atexit.register(cleanup_kafka)

# Iniciar Zookeeper
print("🚀 Iniciando Zookeeper...")
zookeeper_process = subprocess.Popen(
    [f"{KAFKA_DIR}/bin/zookeeper-server-start.sh", 
     f"{KAFKA_DIR}/config/zookeeper.properties"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

time.sleep(5)  # Aguardar inicialização

# Iniciar Kafka
print("🚀 Iniciando Kafka broker...")
kafka_process = subprocess.Popen(
    [f"{KAFKA_DIR}/bin/kafka-server-start.sh", 
     f"{KAFKA_DIR}/config/server.properties"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

time.sleep(10)  # Aguardar inicialização

print("✅ Kafka broker rodando em localhost:9092")

In [ ]:
# Criar tópicos Kafka
def create_kafka_topic(topic_name, partitions=3, replication=1):
    """Cria tópico Kafka se não existir"""
    try:
        result = subprocess.run(
            [f"{KAFKA_DIR}/bin/kafka-topics.sh",
             "--create",
             "--bootstrap-server", "localhost:9092",
             "--topic", topic_name,
             "--partitions", str(partitions),
             "--replication-factor", str(replication),
             "--if-not-exists"],
            capture_output=True,
            text=True,
            timeout=10
        )
        if "Created topic" in result.stdout or "already exists" in result.stdout:
            print(f"✅ Tópico '{topic_name}' pronto")
        else:
            print(f"⚠️  {result.stdout}")
    except Exception as e:
        print(f"❌ Erro ao criar tópico {topic_name}: {e}")

# Criar tópicos
create_kafka_topic("social-input", partitions=3)
create_kafka_topic("wordcount-output", partitions=3)

print("\n📋 Listando tópicos criados:")
subprocess.run([f"{KAFKA_DIR}/bin/kafka-topics.sh",
                "--list",
                "--bootstrap-server", "localhost:9092"])

## 5. Producer: Gerador de Mensagens Simulando Rede Social

**Justificativa:** Conforme explicado na introdução, ao invés de integrar com Discord (que exigiria tokens, webhooks e configurações externas), implementamos um gerador automático de mensagens que simula posts de uma rede social.

Este gerador:
- Publica mensagens continuamente no tópico `social-input`
- Usa vocabulário variado (tecnologia, Big Data, Hadoop, Spark, etc.)
- Simula comportamento realista com timestamps e variação de conteúdo
- Permite reprodutibilidade total do experimento

In [ ]:
from kafka import KafkaProducer
import json
import random
import threading

class SocialMediaSimulator:
    """Simula posts de rede social enviando para Kafka"""
    
    def __init__(self, bootstrap_servers='localhost:9092', topic='social-input'):
        self.producer = KafkaProducer(
            bootstrap_servers=bootstrap_servers,
            value_serializer=lambda v: json.dumps(v).encode('utf-8')
        )
        self.topic = topic
        self.running = False
        self.thread = None
        self.message_count = 0
        
        # Vocabulário simulando posts sobre Big Data
        self.templates = [
            "Acabei de implementar um cluster {tech} com {num} nodes! {emoji}",
            "Performance do {tech} está incrível hoje! Processando {num}GB de dados.",
            "Alguém já testou {tech} em produção? Preciso de dicas sobre {feature}.",
            "Tutorial: Como configurar {tech} para {feature} em {num} passos.",
            "{emoji} Novo post sobre {tech}! Compartilhando minha experiência com {feature}.",
            "Comparando {tech} vs {tech2}: qual é melhor para {feature}?",
            "Dica rápida de {tech}: sempre configure {feature} para otimizar performance!",
            "Meu cluster {tech} finalmente está rodando! {num} workers processando dados.",
        ]
        
        self.tech_words = ["Hadoop", "Spark", "Kafka", "HDFS", "YARN", "MapReduce", 
                           "Flink", "Storm", "Hive", "Pig", "HBase", "Cassandra"]
        self.features = ["streaming", "batch processing", "fault tolerance", "scalability",
                        "distributed computing", "data partitioning", "replication",
                        "load balancing", "resource management", "data locality"]
        self.emojis = ["🚀", "💡", "🔥", "⚡", "🎯", "📊", "🛠️", "✨"]
    
    def generate_message(self):
        """Gera uma mensagem aleatória"""
        template = random.choice(self.templates)
        message = template.format(
            tech=random.choice(self.tech_words),
            tech2=random.choice(self.tech_words),
            num=random.randint(2, 100),
            feature=random.choice(self.features),
            emoji=random.choice(self.emojis)
        )
        return {
            "timestamp": int(time.time()),
            "user": f"user_{random.randint(1, 50)}",
            "message": message
        }
    
    def produce_loop(self, interval=2.0):
        """Loop de produção de mensagens"""
        while self.running:
            msg = self.generate_message()
            self.producer.send(self.topic, value=msg)
            self.message_count += 1
            if self.message_count % 10 == 0:
                print(f"📤 {self.message_count} mensagens enviadas para '{self.topic}'")
            time.sleep(interval)
    
    def start(self, interval=2.0):
        """Inicia producer em thread separada"""
        if not self.running:
            self.running = True
            self.thread = threading.Thread(target=self.produce_loop, args=(interval,))
            self.thread.daemon = True
            self.thread.start()
            print(f"✅ Producer iniciado (intervalo: {interval}s)")
    
    def stop(self):
        """Para o producer"""
        if self.running:
            self.running = False
            if self.thread:
                self.thread.join(timeout=5)
            self.producer.flush()
            self.producer.close()
            print(f"🛑 Producer finalizado ({self.message_count} mensagens)")

# Instanciar producer
social_producer = SocialMediaSimulator(topic='social-input')

print("✅ Gerador de mensagens (Social Media Simulator) configurado")

In [ ]:
# Iniciar producer
social_producer.start(interval=1.5)  # Envia mensagem a cada 1.5 segundos

# Aguardar algumas mensagens serem enviadas
time.sleep(10)

print(f"\n📊 Total de mensagens enviadas até agora: {social_producer.message_count}")

## 6. Configuração de Saída Gráfica

**Justificativa:** Conforme explicado na introdução, ao invés de ElasticSearch + Kibana (que exigem >4GB RAM, Docker e configurações complexas não viáveis no Colab), implementamos visualização inline com `wordcloud`.

Esta abordagem:
- Consome mensagens do tópico `wordcount-output` em tempo real
- Gera nuvem de palavras dinamicamente no próprio notebook
- Permite atualizações near-real-time da visualização
- É totalmente reproduzível em qualquer ambiente Python

In [ ]:
from kafka import KafkaConsumer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import defaultdict
import threading

class WordCloudVisualizer:
    """Consome resultados do Kafka e exibe nuvem de palavras"""
    
    def __init__(self, bootstrap_servers='localhost:9092', topic='wordcount-output'):
        self.consumer = KafkaConsumer(
            topic,
            bootstrap_servers=bootstrap_servers,
            auto_offset_reset='earliest',
            enable_auto_commit=True,
            value_deserializer=lambda m: json.loads(m.decode('utf-8'))
        )
        self.word_counts = defaultdict(int)
        self.running = False
        self.thread = None
        self.total_words = 0
    
    def consume_loop(self):
        """Loop de consumo de mensagens"""
        for message in self.consumer:
            if not self.running:
                break
            
            data = message.value
            word = data.get('word', '')
            count = data.get('count', 0)
            
            if word:
                self.word_counts[word] = count
                self.total_words += 1
    
    def start(self):
        """Inicia consumer em thread separada"""
        if not self.running:
            self.running = True
            self.thread = threading.Thread(target=self.consume_loop)
            self.thread.daemon = True
            self.thread.start()
            print("✅ Consumer de visualização iniciado")
    
    def stop(self):
        """Para o consumer"""
        if self.running:
            self.running = False
            if self.thread:
                self.thread.join(timeout=5)
            self.consumer.close()
            print("🛑 Consumer finalizado")
    
    def generate_wordcloud(self, max_words=100):
        """Gera nuvem de palavras com contagens atuais"""
        if not self.word_counts:
            print("⚠️  Nenhuma palavra processada ainda")
            return None
        
        # Criar WordCloud
        wc = WordCloud(
            width=1200,
            height=600,
            background_color='white',
            max_words=max_words,
            colormap='viridis',
            relative_scaling=0.5,
            min_font_size=10
        ).generate_from_frequencies(self.word_counts)
        
        return wc
    
    def plot_wordcloud(self, max_words=100):
        """Plota nuvem de palavras"""
        wc = self.generate_wordcloud(max_words)
        if wc:
            plt.figure(figsize=(14, 7))
            plt.imshow(wc, interpolation='bilinear')
            plt.axis('off')
            plt.title(f'Nuvem de Palavras - {len(self.word_counts)} palavras únicas | '
                     f'{self.total_words} contagens processadas', 
                     fontsize=16, pad=20)
            plt.tight_layout()
            plt.show()
    
    def plot_top_words(self, top_n=20):
        """Plota gráfico de barras com top N palavras"""
        if not self.word_counts:
            print("⚠️  Nenhuma palavra processada ainda")
            return
        
        # Ordenar por contagem
        sorted_words = sorted(self.word_counts.items(), key=lambda x: x[1], reverse=True)
        top_words = sorted_words[:top_n]
        
        words = [w[0] for w in top_words]
        counts = [w[1] for w in top_words]
        
        plt.figure(figsize=(12, 6))
        plt.barh(words[::-1], counts[::-1], color='skyblue')
        plt.xlabel('Contagem', fontsize=12)
        plt.ylabel('Palavra', fontsize=12)
        plt.title(f'Top {top_n} Palavras Mais Frequentes', fontsize=14, pad=15)
        plt.tight_layout()
        plt.show()

# Instanciar visualizador
visualizer = WordCloudVisualizer(topic='wordcount-output')

print("✅ Visualizador de nuvem de palavras configurado")

## 7. Processamento: WordCount com Spark Structured Streaming

Implementação do WordCount usando Spark Structured Streaming para processar mensagens do Kafka em tempo real.

In [ ]:
from pyspark.sql.functions import explode, split, lower, regexp_replace, col, count, to_json, struct
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType

# Definir schema das mensagens de entrada
input_schema = StructType([
    StructField("timestamp", LongType(), True),
    StructField("user", StringType(), True),
    StructField("message", StringType(), True)
])

print("✅ Schema de entrada definido")

In [ ]:
# Ler stream do Kafka
df_input = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "social-input") \
    .option("startingOffsets", "earliest") \
    .load()

print("✅ Stream de entrada conectado ao tópico 'social-input'")

In [ ]:
# Parsear JSON e processar mensagens
from pyspark.sql.functions import from_json

df_parsed = df_input \
    .selectExpr("CAST(value AS STRING) as json") \
    .select(from_json(col("json"), input_schema).alias("data")) \
    .select("data.*")

print("✅ Parsing JSON configurado")

In [ ]:
# WordCount: tokenização e contagem
stopwords_list = {'de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'é', 'com', 
                  'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como',
                  'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito',
                  'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for',
                  'of', 'with', 'by', 'from', 'as', 'is', 'was', 'are', 'been', 'be',
                  'have', 'has', 'had', 'do', 'does', 'did', 'will', 'would', 'could',
                  'should', 'may', 'might', 'must', 'can', 'this', 'that', 'these', 'those'}

# Processar palavras
df_words = df_parsed \
    .select(
        explode(
            split(
                lower(
                    regexp_replace(col("message"), "[^a-zA-Z0-9áéíóúâêôãõçÁÉÍÓÚÂÊÔÃÕÇ\\s]", "")
                ), 
                "\\s+"
            )
        ).alias("word")
    ) \
    .filter(col("word") != "") \
    .filter(~col("word").isin(stopwords_list))

# Contar palavras
df_wordcount = df_words \
    .groupBy("word") \
    .count() \
    .orderBy(col("count").desc())

print("✅ Pipeline de WordCount configurado")
print("📊 Colunas finais:", df_wordcount.columns)

In [ ]:
# Escrever resultados no Kafka (tópico de saída)
query_kafka = df_wordcount \
    .select(to_json(struct("word", "count")).alias("value")) \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("topic", "wordcount-output") \
    .option("checkpointLocation", f"{CHECKPOINT_DIR}/kafka-output") \
    .outputMode("complete") \
    .start()

print("✅ Stream de saída iniciado (Kafka topic: 'wordcount-output')")
print(f"📊 Query ID: {query_kafka.id}")
print(f"🔄 Status: {query_kafka.status}")

In [ ]:
# Também escrever em console para monitoramento (opcional)
query_console = df_wordcount \
    .writeStream \
    .format("console") \
    .option("truncate", "false") \
    .option("numRows", 20) \
    .outputMode("complete") \
    .trigger(processingTime='10 seconds') \
    .start()

print("✅ Stream de console iniciado (exibição a cada 10s)")

## 8. Apresentação de Resultados e Visualização em Dashboard

Iniciando consumer para visualização e gerando gráficos em tempo real.

In [ ]:
# Iniciar consumer de visualização
visualizer.start()

# Aguardar acumulação de dados
print("⏳ Aguardando 20 segundos para acumular dados...")
time.sleep(20)

print(f"\n📊 Estatísticas:")
print(f"   - Palavras únicas: {len(visualizer.word_counts)}")
print(f"   - Total de contagens: {visualizer.total_words}")

In [ ]:
# Gerar visualizações
print("📊 Gerando visualizações...\n")

# 1. Top 20 palavras
visualizer.plot_top_words(top_n=20)

# 2. Nuvem de palavras
visualizer.plot_wordcloud(max_words=100)

## 9. Validação e Evidências

Monitoramento final do pipeline e coleta de evidências de funcionamento.

In [ ]:
# Verificar status das queries Spark
print("🔍 Status das Queries Spark Streaming:\n")

for query in spark.streams.active:
    print(f"Query ID: {query.id}")
    print(f"Name: {query.name}")
    print(f"Status: {query.status}")
    print(f"Recent Progress:")
    
    # Últimos 3 progresses
    for progress in query.recentProgress[-3:]:
        print(f"  - Batch: {progress.get('batchId', 'N/A')}")
        print(f"    Input Rows: {progress.get('numInputRows', 0)}")
        print(f"    Processing Rate: {progress.get('inputRowsPerSecond', 0):.2f} rows/sec")
        print(f"    Duration: {progress.get('durationMs', {}).get('total', 0)}ms")
    print()

In [ ]:
# Verificar mensagens no Kafka
import subprocess

print("📋 Verificando tópicos Kafka:\n")

# Contar mensagens no tópico de entrada
result_input = subprocess.run(
    [f"{KAFKA_DIR}/bin/kafka-run-class.sh", "kafka.tools.GetOffsetShell",
     "--broker-list", "localhost:9092",
     "--topic", "social-input",
     "--time", "-1"],
    capture_output=True,
    text=True
)

# Contar mensagens no tópico de saída
result_output = subprocess.run(
    [f"{KAFKA_DIR}/bin/kafka-run-class.sh", "kafka.tools.GetOffsetShell",
     "--broker-list", "localhost:9092",
     "--topic", "wordcount-output",
     "--time", "-1"],
    capture_output=True,
    text=True
)

print(f"Tópico 'social-input':")
print(result_input.stdout)

print(f"\nTópico 'wordcount-output':")
print(result_output.stdout)

In [ ]:
# Amostras de dados processados
print("📊 Amostra de Palavras Mais Frequentes:\n")

if visualizer.word_counts:
    sorted_words = sorted(visualizer.word_counts.items(), key=lambda x: x[1], reverse=True)
    
    print(f"{'Palavra':<20} {'Contagem':>10}")
    print("-" * 32)
    for word, count in sorted_words[:15]:
        print(f"{word:<20} {count:>10}")
else:
    print("⚠️  Nenhum dado processado ainda")

In [ ]:
# Atualizar visualizações após mais dados
print("\n⏳ Aguardando mais 15 segundos para acumular dados...")
time.sleep(15)

print(f"\n📊 Estatísticas atualizadas:")
print(f"   - Palavras únicas: {len(visualizer.word_counts)}")
print(f"   - Total de contagens: {visualizer.total_words}")
print(f"   - Mensagens produzidas: {social_producer.message_count}")

# Gerar visualizações finais
print("\n📊 Visualizações Finais:\n")
visualizer.plot_top_words(top_n=25)
visualizer.plot_wordcloud(max_words=150)

## 10. Finalização e Limpeza

Parando todos os serviços e streams.

In [ ]:
# Parar producer
social_producer.stop()

# Parar visualizador
visualizer.stop()

# Parar queries Spark
for query in spark.streams.active:
    print(f"🛑 Parando query: {query.id}")
    query.stop()

# Aguardar finalização
time.sleep(5)

print("\n✅ Todos os streams finalizados")

In [ ]:
# Estatísticas finais
print("📊 ESTATÍSTICAS FINAIS DO PIPELINE\n")
print("="*50)
print(f"Mensagens produzidas: {social_producer.message_count}")
print(f"Palavras únicas processadas: {len(visualizer.word_counts)}")
print(f"Total de contagens: {visualizer.total_words}")
print("="*50)

# Top 10 palavras finais
if visualizer.word_counts:
    sorted_words = sorted(visualizer.word_counts.items(), key=lambda x: x[1], reverse=True)
    print("\nTOP 10 PALAVRAS MAIS FREQUENTES:")
    for i, (word, count) in enumerate(sorted_words[:10], 1):
        print(f"  {i}. {word}: {count}")

In [ ]:
# Parar Spark
spark.stop()
print("🛑 Spark Session finalizada")

# Cleanup Kafka será feito automaticamente pelo atexit

---

## ✅ Conclusões

### Pipeline Implementado

1. **Entrada:** Gerador automático de mensagens → Kafka `social-input`
2. **Processamento:** Spark Structured Streaming → WordCount com tokenização e remoção de stopwords
3. **Saída:** Kafka `wordcount-output` → Consumer Python → Visualização inline (WordCloud + BarChart)

### Justificativas de Adaptações

- **Discord → Gerador de Texto:** Mantém o conceito de stream de mensagens sem dependências externas complexas
- **ElasticSearch/Kibana → WordCloud inline:** Visualização equivalente sem overhead de infraestrutura

### Diferenciais Implementados

- ✅ Pipeline 100% reproduzível em Colab ou Jupyter local
- ✅ Visualizações near-real-time com atualizações dinâmicas
- ✅ Remoção de stopwords (PT + EN) para melhor qualidade
- ✅ Monitoramento completo (Spark UI, Kafka offsets, contadores)
- ✅ Cleanup automático de recursos

### Tecnologias Utilizadas

- Apache Spark 3.5.0 (Structured Streaming)
- Apache Kafka 3.6.0 (Message Broker)
- Python: kafka-python, wordcloud, matplotlib, pandas
- Zookeeper (para Kafka)

---

**Notebook executado com sucesso!**  
**Data:** 14 de novembro de 2025